# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
 
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_fscore_support,accuracy_score,label_ranking_average_precision_score
from sklearn.model_selection  import GridSearchCV, RandomizedSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
import lightgbm as lgb

import re

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords


In [ ]:
# # load data from database
# engine = create_engine('sqlite:///InsertDatabaseName.db')
# df = 
# X = 
# Y = # load data from database
engine = create_engine('sqlite:///DisasterResponseJupter.db')

inspector = inspect(engine)

# Get table information
print(inspector.get_table_names())
df = pd.read_sql("SELECT * FROM Message", engine)
# X = 
# Y = 
df.head()

In [ ]:
X_names = ['message']
X = df['message'].values
X

In [ ]:
Y_names = list(set(df.columns.values) - set( ['id','message', 'original', 'genre']))
Y = df[Y_names]
Y

In [ ]:
category_names = Y.columns.values
category_names

In [ ]:
!free -h

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    #init stopwords and WordNetLemmatizer
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
     
    # tokenize text
    tokens = word_tokenize(text)
     
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
 
    return tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:

pipeline =  Pipeline([
    ('tfidfvect', TfidfVectorizer(tokenizer=tokenize, stop_words='english')),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
X_train

In [ ]:
y_train

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_preds = pd.DataFrame(pipeline.predict(X_test), columns = category_names)
y_preds.head()

In [ ]:
y_test.shape, y_preds.shape

In [ ]:
def evaluate_mode(model, X_test, y_test, col_name):
    """ Evaluate model on test set,
        Predict results for each category.
        
    Args:
        model: trained model
        X_test: pandas.DataFrame for predict 
        y_test: pandas.DataFrame for labeled test set
        col_name: list for category names
        `
    Returns: 
        none.
    
    """
    
    # predict test df
    Y_pred = model.predict(X_test)
    tot_acc = 0
    tot_f1 = 0
    # print report 
    for i, cat in enumerate(col_name):    
        metrics =  classification_report(y_test[y_test.columns[i]], Y_pred[:,i])
        tot_acc += accuracy_score(y_test[y_test.columns[i]], Y_pred[:,i])
        tot_f1 += precision_recall_fscore_support(y_test[y_test.columns[i]], Y_pred[:,i], average = 'weighted')[2]
        print(cat, 'accuracy: {:.5f}'.format(accuracy_score(y_test[y_test.columns[i]], Y_pred[:,i])))
        print(metrics)
    print('total accuracy {:.5f}'.format(tot_acc/len(col_name)))
    print('total f1 {:.5f}'.format(tot_f1/len(col_name)))


In [ ]:
evaluate_mode(pipeline, X_test, y_test, category_names)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
        'tfidfvect__ngram_range': [(1, 1), (1, 2)],
        'tfidfvect__max_df': [0.75, 1.0],
        'clf__estimator__n_estimators': [10, 15],
        }

cv = RandomizedSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv_model = cv.fit(X_train, y_train)


In [ ]:
evaluate_mode(cv_model, X_test, y_test, Y.columns)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
estimator = lgb.LGBMRegressor(num_leaves=31)


In [ ]:
class TextLengthExtractor(BaseEstimator, TransformerMixin):
    '''
    Add the length of the text message as a feature to dataset
    
    The assumption is people who is in urgent disaster condition will prefer to use less words to express
    '''
    
    def fit(self, x, y=None):
        return self

    def transform(self, X):
        return pd.DataFrame(X).applymap(len)



In [ ]:
# pipeline_2 = Pipeline([
#     ('features', FeatureUnion([       
#         ('tfidfvect', TfidfVectorizer(tokenizer=tokenize)),
#         ('text_total_length', TextLengthExtractor()) # Add a new feature
#         ])),
#     ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=42))) # Change the estimator to AdaBoost
#     ])

In [ ]:
pipeline_2 = Pipeline([
    ('features', FeatureUnion([       
        ('tfidfvect', TfidfVectorizer(tokenizer=tokenize)),
        ('text_total_length', TextLengthExtractor()) # Add a new feature
        ])),
    ('clf', MultiOutputClassifier( lgb.LGBMClassifier(num_leaves=70))) # Change the estimator to LGBMClassifier
    ])

In [ ]:
parameters_2 = {
        'features__tfidfvect__ngram_range': [(1, 1), (1, 2)],
        'features__tfidfvect__max_df': [0.75, 1.0],
        'features__tfidfvect__max_features': [2000, 5000],
        'features__transformer_weights': (
            {'tfidfvect': 1, 'text_total_length': 0.5},
            {'tfidfvect': 1, 'text_total_length': 1}
        ),
        'clf__estimator__boosting_type': ['gbdt', 'dart'],
        'clf__estimator__n_estimators': [10, 15, 25],
        'clf__estimator__learning_rate': [0.5, 0.1]
        }

cv_2 = RandomizedSearchCV(pipeline_2, param_grid=parameters_2, cv=5)

In [ ]:
cv_2_model = cv_2.fit(X_train, y_train)


In [ ]:
evaluate_mode(cv_2_model, X_test, y_test, Y.columns)



### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.